In [29]:
pip install jupyter_argparser

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement jupyter_argparser (from versions: none)
ERROR: No matching distribution found for jupyter_argparser
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [27]:
# import the necessary packages
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--dataset", required=True,
	help="path to input directory of faces + images")
ap.add_argument("-e", "--embeddings", required=True,
	help="path to output serialized db of facial embeddings")
ap.add_argument("-d", "--detector", required=True,
	help="path to OpenCV's deep learning face detector")
ap.add_argument("-m", "--embedding-model", required=True,
	help="path to OpenCV's deep learning face embedding model")
ap.add_argument("-c", "--confidence", type=float, default=0.5,
	help="minimum probability to filter weak detections")
# args = vars(ap.parse_args())
# args = parser.parse_args(['-i', '/'])
args = argparse.Namespace(verbose=False, verbose_1=False)
args

Namespace(verbose=False, verbose_1=False)

In [17]:
import argparse
parser = argparse.ArgumentParser()
parser.parse_args()

usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/albena/.local/share/jupyter/runtime/kernel-199b7fd6-a3dd-45ec-998c-41fc2f37c433.json


SystemExit: 2